# Identify new construction buildings that are probably rent stabilized

In [100]:
import re
import os
import json
import pandas as pd
import requests
import folium
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 200)

In [101]:
app_id = os.environ['nyc_geoclient_app_id']
app_key = os.environ['nyc_geoclient_app_key']

mn_file = "data/2018-HCR-Manhattan.csv"
bk_file = "data/2018-HCR-Brooklyn.csv"

In [102]:
pluto_raw = pd.read_csv("data/nyc_pluto_21v1_csv/pluto_21v1.csv", low_memory=False)

In [103]:
pluto = pluto_raw[["borough", "block", "lot", "latitude", "longitude", "unitsres", "yearbuilt"]].copy()

In [104]:
mn_raw = pd.read_csv(mn_file)

In [105]:
bk_raw = pd.read_csv(bk_file)

In [106]:
assert sum(bk_raw["STATUS3 BLOCK LOT"] == '421-A') == 0
assert sum(bk_raw["Unnamed: 0"] == '421-A') == 0
assert sum(bk_raw["BLOCK LOT"] == '421-A') == 0

In [120]:
def get_block_lot(x):
    x['COUNTY STATUS1'] = x['COUNTY STATUS1'].replace("61 ", "")
    x['COUNTY STATUS1'] = x['COUNTY STATUS1'].replace("62 ", "")
    x['STSUFX2 CITY'] = 'MANHATTAN' if 'NEW YORK' in x['STSUFX2 CITY'] else 'BROOKLYN'
    if not isinstance(x["BLOCK LOT"], str):
        x["BLOCK LOT"] = f"{x['STATUS3 BLOCK LOT']} {int(x['Unnamed: 0'])}"
    bbls = x["BLOCK LOT"].split()
    if len(bbls) > 2:
        bbl_regex = re.compile(r'[\d]+\s[\d]+$')
        bbl = bbl_regex.search(x["BLOCK LOT"])
        x["BLOCK LOT"] = bbl.group()
        x["STATUS3"] = x["BLOCK LOT"].replace(bbl.group(), "")
    x["block"], x["lot"] = x["BLOCK LOT"].split()
    return x

In [121]:
def label_421a(x):
    statuses = [str(x.status1), str(x.status2), str(x.status3)]
    try:
        is421a = any("421-A" in s for s in statuses)
    except TypeError:
        print(x)
    return is421a

In [122]:
def clean_up(df):
    """ Clean up after tabula processing """
    keep_cols = ['ZIP', 'BLDGNO1', 'STREET1', 'STSUFX1 BLDGNO2', 'STSUFX2 CITY', 'COUNTY STATUS1', 'STATUS2', 'STATUS3', 'block', 'lot']
    name_cols = ["zip", "bldgno", "street", "stsufx", "boro", "status1", "status2", "status3", "block", "lot"]
    df = df.apply(get_block_lot, axis=1)
    df = df[keep_cols]
    df.columns = name_cols
    df["is421a"] = df.apply(label_421a, axis=1)
    df.block = df.block.astype(int)
    df.lot = df.lot.astype(int)
    return df

In [123]:
mn = clean_up(mn_raw)
mn = mn.merge(pluto[pluto.borough=='MN'], how="left", on=["block", "lot"])
mn.sample(5)

,zip,bldgno,street,stsufx,boro,status1,status2,status3,block,lot,is421a,borough,latitude,longitude,unitsres,yearbuilt
3828,10014,65,JANE,ST,MANHATTAN,MULTIPLE DWELLING A,NaN,NaN,626,1,False,MN,40.738327,-74.006225,27.0,1901.0
257,10002,101,DELANCEY,ST,MANHATTAN,MULTIPLE DWELLING A,NaN,NaN,409,43,False,MN,40.718487,-73.988907,20.0,1920.0
11128,10033,100,BENNETT,AVE,MANHATTAN,MULTIPLE DWELLING A,NaN,NaN,2180,169,False,MN,40.853371,-73.935596,62.0,1939.0
5998,10023,34 TO 40,W 72ND,ST,MANHATTAN,MULTIPLE DWELLING A,NON-EVICT COOP/CONDO,NaN,1124,7501,False,MN,40.776743,-73.977939,142.0,1926.0
4751,10019,250,W 50TH,ST,MANHATTAN,MULTIPLE DWELLING A,421-A,NaN,1021,1,True,MN,40.761826,-73.985817,550.0,1998.0


In [124]:
bk = clean_up(bk_raw)
bk = bk.merge(pluto[pluto.borough=='BK'], how="left", on=["block", "lot"])
bk.sample(5)

,zip,bldgno,street,stsufx,boro,status1,status2,status3,block,lot,is421a,borough,latitude,longitude,unitsres,yearbuilt
8024,11220,517,49TH,ST,BROOKLYN,MULTIPLE DWELLING A,GARDEN COMPLEX,ARTICLE 11,775,80,False,BK,40.645553,-74.009099,8.0,1914.0
2636,11208,64,MONTAUK,AVE,BROOKLYN,MULTIPLE DWELLING A,NaN,NaN,3975,1,False,BK,40.678126,-73.879411,102.0,1931.0
4092,11211,236A,SKILLMAN,AVE,BROOKLYN,MULTIPLE DWELLING A,421-A,NaN,2893,21,True,BK,40.716046,-73.941309,3.0,2001.0
12093,11229,3203,NOSTRAND,AVE,BROOKLYN,MULTIPLE DWELLING A,NaN,NaN,6836,50,False,BK,40.605858,-73.942576,108.0,1958.0
15397,11238,403,SAINT JOHNS,PL,BROOKLYN,MULTIPLE DWELLING A,NaN,NaN,1173,67,False,BK,40.674231,-73.964494,27.0,1921.0


In [129]:
# Apply search criteria
df = mn.append(bk)
df = df[df.is421a & (df.yearbuilt >= 2009) & (df.unitsres >= 10)].copy()
df.unitsres = df.unitsres.astype(int)
df.yearbuilt = df.yearbuilt.astype(int)

In [130]:
print(len(df))

300


In [131]:
df.sample(5)

,zip,bldgno,street,stsufx,boro,status1,status2,status3,block,lot,is421a,borough,latitude,longitude,unitsres,yearbuilt
6463,11216,194,HERKIMER,ST,BROOKLYN,MULTIPLE DWELLING A,GARDEN COMPLEX,421-A,1867,47,True,BK,40.679322,-73.947085,10,2014
13827,11235,422 TO 424,OCEAN VIEW,AVE,BROOKLYN,MULTIPLE DWELLING A,421-A,NaN,8674,7,True,BK,40.579118,-73.963230,16,2016
5808,11215,585 TO 593,6TH,AVE,BROOKLYN,MULTIPLE DWELLING A,421-A,COOP/CONDO PLAN FILE,1054,10,True,BK,40.663279,-73.987521,27,2009
3929,11211,199 TO 225,N 9TH,ST 220,BROOKLYN,MULTIPLE DWELLING A,421-A,NaN,2306,1,True,BK,40.718059,-73.954673,113,2011
8825,11221,867 TO 869,DEKALB,AVE,BROOKLYN,MULTIPLE DWELLING A,GARDEN COMPLEX,421-A,1777,58,True,BK,40.692708,-73.941784,35,2015


In [142]:
def plot_bldg(x, this_map):
    folium.CircleMarker(location=[x['latitude'], x['longitude']], radius=2, weight=2, opacity=.8,
                       tooltip=f"{x['bldgno']} {x['street']} {x['stsufx']} - {x['block']} {x['lot']}<br>{x['unitsres']} units built in {x['yearbuilt']}").add_to(this_map)
    
nyc_map = folium.Map(prefer_canvas=True, tiles='Stamen Toner')
df[df.latitude.notnull()].apply(plot_bldg, axis=1, this_map=nyc_map)
nyc_map.fit_bounds([[40.65, -74.1], [40.82, -73.8]]) # nyc_map.get_bounds()
nyc_map.save('nyc-421-a.html')

In [137]:
# Don't need to do this anymore because lat/lon is in pluto csv
def get_lat_lon(x):
    boro = x['borough'].lower()
    block = x['block']
    lot = x['lot']
    url = f"https://api.cityofnewyork.us/geoclient/v1/bbl.json?borough={boro}&block={block}&lot={lot}&app_id={app_id}&app_key={app_key}"
    r = requests.get(url)
    try:
        record = r.json()['bbl']
        x['lat'] = record['latitudeInternalLabel']
        x['lon'] = record['longitudeInternalLabel']
    except:
        print(f"Lat/lon not found for {block} {lot}")
    return x

# mn = mn.apply(get_lat_lon, axis=1)
# bk = bk.apply(get_lat_lon, axis=1)